In [ ]:
#firstly, we import pandas in order to work with the data set
import pandas as pd
import numpy as np

#First step from creating the model is to import the important library for machine learning, which in our situation is scikit-learn.
import sklearn as scikit_learn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

In [561]:
#secondly, we have to configure the file path in order to load the data f
# Load the dataset
file_path = "Data/set8_it1.csv"  # Update with your file path
df = pd.read_csv(file_path)

In [562]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2363 entries, 0 to 2362
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   DI              2363 non-null   int64  
 1   TradeValue      2363 non-null   float64
 2   RealmType       2363 non-null   object 
 3   Enclave         2363 non-null   object 
 4   LivingQuarters  2244 non-null   float64
 5   ParcelSize      2363 non-null   float64
 6   ParcelSizeUnit  2363 non-null   object 
dtypes: float64(3), int64(1), object(3)
memory usage: 129.4+ KB


In [563]:
# 1. Remove negative values in ParcelSize
df = df[df["ParcelSize"] >= 10]

In [564]:
# 2. Remove rows with missing LivingQuarters data
df["LivingQuarters"] = df["LivingQuarters"].fillna(df["LivingQuarters"].median())

In [565]:
# 3. Remove rows where ParcelSize is less than 1
#df = df[df["ParcelSize"] >= 1]

In [566]:
# 4. Remove duplicate rows
df = df.drop_duplicates()

In [567]:
# 5. Standardize Enclave column (lowercase and strip spaces)
df["Enclave"] = df["Enclave"].str.strip().str.lower()
# Define a mapping to correct similar enclave names
enclave_corrections = {
    "shadowmeres": "shadowmere",
    "mystvales": "mystvale",
    "dragonspires": "dragonspire",
    "stormwatchs": "stormwatch",
    "thornfields": "thornfield",
    "petalbridges": "petalbridge",
    "silverglens": "silverglen",
    "riverbends": "riverbend",
    "emberfalls": "emberfall"
}

# Replace incorrect names with correct ones
df["Enclave"] = df["Enclave"].replace(enclave_corrections)

# Verify the results
df["Enclave"].unique()

array(['shadowmere', 'ravenstone', 'petalbridge', 'emberfall',
       'dragonspire', 'riverbend', 'cloudrest', 'sunspire', 'nighthaven',
       'brightforge', 'wyverncliff', 'crystalhollow', 'wolfsbane',
       'duskwood', 'mystvale', 'glimmerwood', 'stormhold', 'starfall',
       'silverglen', 'thornfield', 'dreamweaver', 'stormwatch',
       'windwhisper', 'oakenshade', 'mossheart', 'verdantia', 'moonglade'],
      dtype=object)

In [568]:
# 6. Convert ParcelSize to sqft if it is in sqm
conversion_factor = 10.7639  # 1 sqm = 10.7639 sqft
df.loc[df["ParcelSizeUnit"] == "sqm", "ParcelSize"] *= conversion_factor
df.loc[df["ParcelSizeUnit"] == "sqm", "ParcelSizeUnit"] = "sqft"  # Standardizing to sqft

In [569]:
# 7. Investigate TradeValue minimum
min_trade_value = df["TradeValue"].min()
outlier_trade_value = df[df["TradeValue"] == min_trade_value]

In [570]:
# 8. Detect outliers using 1.5*IQR method
def detect_outliers(column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] < lower_bound) & (df[column] > upper_bound)]

In [571]:
outliers_trade_value = detect_outliers("TradeValue")
outliers_living_quarters = detect_outliers("LivingQuarters")
outliers_parcel_size = detect_outliers("ParcelSize")

# Get index values of all outliers
outlier_indices = (
    outliers_trade_value.index.to_list() + 
    outliers_living_quarters.index.to_list() + 
    outliers_parcel_size.index.to_list()
)

# Drop these rows from the dataset
df = df.drop(index=outlier_indices)

In [572]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2318 entries, 0 to 2362
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   DI              2318 non-null   int64  
 1   TradeValue      2318 non-null   float64
 2   RealmType       2318 non-null   object 
 3   Enclave         2318 non-null   object 
 4   LivingQuarters  2318 non-null   float64
 5   ParcelSize      2318 non-null   float64
 6   ParcelSizeUnit  2318 non-null   object 
dtypes: float64(3), int64(1), object(3)
memory usage: 144.9+ KB


In [573]:
df

,DI,TradeValue,RealmType,Enclave,LivingQuarters,ParcelSize,ParcelSizeUnit
0,0,125000.0,Meadowlands,shadowmere,1032.0,9280.0,sqft
1,1,250000.0,Meadowlands,ravenstone,1933.0,11170.0,sqft
2,2,130000.0,GnomeBurough,petalbridge,1012.0,5500.0,sqft
3,3,157500.0,Meadowlands,emberfall,843.0,13014.0,sqft
4,4,156000.0,Meadowlands,dragonspire,807.0,8092.0,sqft
...,...,...,...,...,...,...,...
2358,2335,135000.0,Meadowlands,dragonspire,1110.5,8064.0,sqft
2359,2336,334000.0,Meadowlands,glimmerwood,2173.0,11025.0,sqft
2360,2337,214000.0,Meadowlands,dragonspire,1138.0,8197.0,sqft
2361,2338,129500.0,Meadowlands,emberfall,759.0,9100.0,sqft


In [574]:
# The next step is to connect the prepared data from above and to use it for getting the mean and median of the TradeValue variable. 
mean_trade_value = df['TradeValue'].mean()
median_trade_value = df['TradeValue'].median()

In [ ]:
#After we have connected the data, we are transforming the categorical data into numerical using the label encoder so the model can fit all of that information so it can make predictions
label_encoder_realm = LabelEncoder()
label_encoder_enclave = LabelEncoder()
df['RealmType'] = label_encoder_realm.fit_transform(df['RealmType'])
df['Enclave'] = label_encoder_enclave.fit_transform(df['Enclave'])

In [577]:
#After that we separate our data into two categories: categorical data, which we transform and data we want to predict. They are separated in X, y respectively
X = df.drop(['TradeValue', 'ParcelSizeUnit'], axis=1)
y = df['TradeValue']

In [578]:
#Then we assign the data, where we are using 80% of the data for training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)   


In [579]:
#The final step is we have created the model, fit in the data and trained it to make predictions
value = y_train.median()
pred_train = [value] * len(y_train)
pred_test = [value] * len(y_test)

In [580]:
#After the predictions we are using metrics such as Mean Absolute Error and Mean squared error to see what the efficiency of our model is
mae = mean_absolute_error(y_test, pred_test) 
mse = mean_squared_error(y_test, pred_test)

In [581]:
mae

54498.995689655174

In [582]:
mean_absolute_percentage_error(y_test, pred_test)

0.297256762750289

In [583]:
mse

6533157540.837644

In [584]:
df2 = df.copy()
df2 = df2.drop(columns=["ParcelSizeUnit"])
df2.corr()

,DI,TradeValue,LivingQuarters,ParcelSize,RealmType_AetherIsles,RealmType_ArcaneMarket,RealmType_CitadelQuarter,RealmType_GnomeBurough,RealmType_Meadowlands,Enclave_brightforge,...,Enclave_silverglen,Enclave_starfall,Enclave_stormhold,Enclave_stormwatch,Enclave_sunspire,Enclave_thornfield,Enclave_verdantia,Enclave_windwhisper,Enclave_wolfsbane,Enclave_wyverncliff
DI,1.000000,-0.013513,-0.038318,-0.003256,-0.022226,-0.015078,-0.014958,0.008969,0.009959,-0.028587,...,0.038561,-0.005975,-0.001587,-0.001053,-0.007430,0.020282,0.026842,0.011860,-0.002136,-0.023617
TradeValue,-0.013513,1.000000,0.688021,0.280705,0.098747,-0.121960,-0.055734,-0.291689,0.244519,0.004964,...,0.445965,-0.096088,-0.075548,-0.141847,0.072749,0.057412,0.003050,0.046811,-0.120750,0.153837
LivingQuarters,-0.038318,0.688021,1.000000,0.316218,0.022784,-0.072958,-0.006305,-0.148523,0.136111,0.054269,...,0.222206,-0.076832,0.029269,-0.102526,-0.006923,0.082652,-0.037471,0.069896,-0.089897,0.042832
ParcelSize,-0.003256,0.280705,0.316218,1.000000,-0.110437,-0.025366,-0.033324,-0.223058,0.265184,-0.002592,...,0.042312,-0.119766,-0.043778,-0.084095,-0.004753,0.041884,-0.043693,0.284710,-0.127549,-0.088577
RealmType_AetherIsles,-0.022226,0.098747,0.022784,-0.110437,1.000000,-0.020291,-0.020291,-0.097275,-0.416072,-0.047145,...,-0.054627,-0.023306,-0.029577,-0.044339,-0.071184,-0.041926,-0.011370,-0.027636,-0.025558,0.860895
RealmType_ArcaneMarket,-0.015078,-0.121960,-0.072958,-0.025366,-0.020291,1.000000,-0.008264,-0.039620,-0.169464,-0.019202,...,-0.022249,-0.009492,0.024695,-0.018059,-0.028993,-0.017076,-0.004631,-0.011256,-0.010410,-0.023570
RealmType_CitadelQuarter,-0.014958,-0.055734,-0.006305,-0.033324,-0.020291,-0.008264,1.000000,-0.039620,-0.169464,0.099113,...,-0.022249,-0.009492,0.171662,-0.018059,-0.028993,0.035666,-0.004631,-0.011256,-0.010410,-0.023570
RealmType_GnomeBurough,0.008969,-0.291689,-0.148523,-0.223058,-0.097275,-0.039620,-0.039620,1.000000,-0.812415,-0.092055,...,-0.101563,0.239586,-0.057751,0.240088,-0.094233,-0.029928,-0.022202,-0.053962,0.262740,-0.112993
RealmType_Meadowlands,0.009959,0.244519,0.136111,0.265184,-0.416072,-0.169464,-0.169464,-0.812415,1.000000,0.087701,...,0.126808,-0.194643,0.023370,-0.180641,0.131733,0.043688,0.027328,0.066422,-0.213454,-0.329837
Enclave_brightforge,-0.028587,0.004964,0.054269,-0.002592,-0.047145,-0.019202,0.099113,-0.092055,0.087701,1.000000,...,-0.051695,-0.022055,-0.027989,-0.041959,-0.067363,-0.039676,-0.010760,-0.026153,-0.024186,-0.054763
